<a href="https://colab.research.google.com/github/24p11/recode-scenario/blob/main/generate_clinical_reports.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import re
import ast
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
scenario_file_name = "generated_scenarios_20250922_1207.csv"
result_file_name = "test_20250922"

In [3]:
nb_samples = 12

In [4]:
path_ref = "/content/drive/MyDrive/Results/clinical_reports/"

In [5]:
df_scenarios= pd.read_csv("/content/drive/MyDrive/Scenario/data/" + scenario_file_name,index_col=0)

In [11]:
#!pip install mistralai

In [12]:
import requests
import time
import logging
from tenacity import retry, stop_after_attempt, wait_exponential


In [1]:
import os
from mistralai import Mistral

In [14]:
api_url = "https://api.mistral.ai/v1/chat"

model = "mistral-large-latest"

In [15]:
client = Mistral(api_key=api_key)

In [16]:
from io import BytesIO
from mistralai import File
import json
import time

In [17]:
import argparse
import json
import os
import random
import time
from io import BytesIO

import httpx
from mistralai import File, Mistral


def create_client():
    """
    Create a Mistral client using the API key from environment variables.

    Returns:
        Mistral: An instance of the Mistral client.
    """
    return Mistral(api_key=os.environ["MISTRAL_API_KEY"])

def generate_random_string(start, end):
    """
    Generate a random string of variable length.

    Args:
        start (int): Minimum length of the string.
        end (int): Maximum length of the string.

    Returns:
        str: A randomly generated string.
    """
    length = random.randrange(start, end)
    return ' '.join(random.choices('abcdefghijklmnopqrstuvwxyz', k=length))

def print_stats(batch_job):
    """
    Print the statistics of the batch job.

    Args:
        batch_job: The batch job object containing job statistics.
    """
    print(f"Total requests: {batch_job.total_requests}")
    print(f"Failed requests: {batch_job.failed_requests}")
    print(f"Successful requests: {batch_job.succeeded_requests}")
    print(
        f"Percent done: {round((batch_job.succeeded_requests + batch_job.failed_requests) / batch_job.total_requests, 4) * 100}")


def create_input_file(client, data):
    """
    Create an input file for the batch job.

    Args:
        client (Mistral): The Mistral client instance.
        num_samples (int): Number of samples to generate.

    Returns:
        File: The uploaded input file object.
    """
    buffer = BytesIO()
    for idx in range(len(data)):
        request = {
            "custom_id": str(idx),
            "body": {
                "max_tokens":  128_000,
                "messages":  [
                    {
                        "role": "system",
                        "content": data.iloc[idx]["system_prompt"]
                        },
                    {
                        "role": "user",
                        "content": data.iloc[idx]["user_prompt"]
                        },
                    {
                        "role": "assistant",
                        "content": data.iloc[idx]["prefix"],
                        "prefix": True
                        },
                    ]
            }
        }
        buffer.write(json.dumps(request).encode("utf-8"))
        buffer.write("\n".encode("utf-8"))
    return client.files.upload(file=File(file_name="file.jsonl", content=buffer.getvalue()), purpose="batch")


def run_batch_job(client, input_file, model):
    """
    Run a batch job using the provided input file and model.

    Args:
        client (Mistral): The Mistral client instance.
        input_file (File): The input file object.
        model (str): The model to use for the batch job.

    Returns:
        BatchJob: The completed batch job object.
    """
    batch_job = client.batch.jobs.create(
        input_files=[input_file.id],
        model=model,
        endpoint="/v1/chat/completions",
        metadata={"job_type": "testing"}
    )

    while batch_job.status in ["QUEUED", "RUNNING"]:
        batch_job = client.batch.jobs.get(job_id=batch_job.id)
        #print_stats(batch_job)
        time.sleep(1)

    print(f"Batch job {batch_job.id} completed with status: {batch_job.status}")
    return batch_job


def download_file(client, file_id, output_path):
    """
    Download a file from the Mistral server.

    Args:
        client (Mistral): The Mistral client instance.
        file_id (str): The ID of the file to download.
        output_path (str): The path where the file will be saved.
    """
    if file_id is not None:
        print(f"Downloading file to {output_path}")
        output_file = client.files.download(file_id=file_id)
        with open(output_path, "wb") as f:
            for chunk in output_file.stream:
                f.write(chunk)
        print(f"Downloaded file to {output_path}")

In [18]:


for i in range(0, int(len(df_scenarios)/nb_samples)):
  print(str(i*nb_samples) + " - "+ str((i+1)*nb_samples) )
  prompts = df_scenarios[i*nb_samples:(i+1)*nb_samples].reset_index(drop=True)
  input_file = create_input_file(client, prompts)
  print(f"Created input file {input_file}")
  batch_job = run_batch_job(client, input_file, model)
  print(f"Job duration: {batch_job.completed_at - batch_job.created_at} seconds")
  download_file(client, batch_job.output_file, path_ref + result_file_name+"/batch_" +  str(i) + ".json" )
  prompts.to_csv(path_ref + result_file_name+"/batch_" +  str(i) + ".csv" )

0 - 12
Created input file id='b4b8c8dd-15cb-4117-b9e3-5681e0d733d2' object='file' size_bytes=104998 created_at=1758535859 filename='file.jsonl' purpose='batch' sample_type='batch_request' source='upload' num_lines=12 mimetype='application/jsonl' signature='5f86690de80ba355'
Batch job e47f57fa-ce9c-46d6-9650-a9705e2d9163 completed with status: SUCCESS
Job duration: 109 seconds
Downloaded file to /content/drive/MyDrive/Results/clinical_reports/test_20250922/batch_0.json


In [19]:
import os
import json

df_res_final=pd.DataFrame()

p = re.compile("```")

output_dir = path_ref + "/"+result_file_name + "/"
for i in range(int(len(df_scenarios)/nb_samples) ): # Iterate through all batch files
    file_path = os.path.join(output_dir, f"batch_{i}.json")
    if os.path.exists(file_path):
        results = []
        with open(file_path, "r", encoding="utf-8") as f:
            for line in f:
                results.append(json.loads(line.strip()))

        # Assuming extract_response is defined and works correctly
        # and results contains the data structure as seen in the previous outputs

        prep_dict=[]
        for result_item in results:
            # Check if the item has a response and extract it
            if "response" in result_item and result_item["response"] and "body" in result_item["response"] and "choices" in result_item["response"]["body"]:
                message_content=result_item["response"]["body"]["choices"][0]["message"]["content"]
                if len(message_content)<40:
                  continue
                m = p.finditer(message_content)
                positions = [i.span() for i in m]
                if len(positions) == 1:
                  res = message_content[positions[0][1]+4:]
                else:
                  res = message_content[positions[0][1]+4:positions[1][0]]

                prep_dict.append({"bacth":i,"num_in_":result_item['custom_id'],"crh":res})

        df_scenarios_save = pd.read_csv(path_ref + result_file_name+"/batch_" +  str(i) + ".csv",index_col=0)
        df_res_tmp = pd.DataFrame(prep_dict)
        df_res_tmp["num_in_"]=  df_res_tmp["num_in_"].astype(int)
        df_res_tmp = df_scenarios_save.merge(df_res_tmp,right_on="num_in_",left_index=True)
        df_res_final = pd.concat([df_res_final,df_res_tmp])



In [22]:
df_res_final.to_csv(path_ref + result_file_name + "/crh_"+result_file_name+".csv")

In [41]:
print(df_res_final.iloc[1,len(df_res_final.columns)-7])
print(df_res_final.iloc[1,len(df_res_final.columns)-1])

**SCÉNARIO DE DÉPART :**
- Âge du patient : 53 ans
- Sexe du patient : Masculin
- Date d'entrée : 27/03/2024
- Date de sortie : 06/04/2024
- Date de naissance : 10/10/1970
- Prénom du patient : Germain
- Nom du patient : Atamian
- Contexte de l'hospitalisation : Prise en charge chirugicale en hospitalisation complète pour - exérèse de lésion d'un ventricule latéral cérébral, par craniotomie (abfa005)
. HOSPITALISATION POUR TRAITEMENT.  La situation est celle d’un patient atteint d’une affection connue, diagnostiquée avant l’admission, hospitalisé pour le traitement de celle-ci. Les circonstances du diagnostic préalable n’importent pas : le diagnostic de l’affection a pu être fait par un médecin généraliste ou spécialiste « de ville », par un service médical d’urgence et de réanimation (SMUR), lors du passage dans une structure d’accueil des urgences, lors d’un séjour précédent dans une autre unité médicale, y compris l’unité d’hospitalisation de courte durée, du même établissement de s

Index(['age', 'cage', 'cage2', 'sexe', 'date_entry', 'date_discharge',
       'date_of_birth', 'first_name', 'last_name', 'icd_primary_code',
       'icd_primary_description', 'icd_parent_code', 'case_management_type',
       'case_management_type_description', 'case_management_type_text',
       'drg_parent_code', 'drg_parent_description', 'icd_secondaray_code',
       'text_secondary_icd_official', 'procedure', 'text_procedure',
       'admission_type', 'admission_mode', 'discharge_disposition', 'dms',
       'los_mean', 'los_sd', 'cancer_stage', 'score_TNM', 'histological_type',
       'treatment_recommandation', 'chemotherapy_regimen', 'biomarkers',
       'first_name_med', 'last_name_med', 'department', 'hospital',
       'cd_md_pec', 'user_prompt', 'system_prompt', 'prefix', 'prefix_len',
       'bacth', 'num_in_', 'crh'],
      dtype='object')

In [21]:
df_res_final.crh

,crh
0,"\n{\n ""CR"": ""\n---\n**Hôpital Larrey - Centre..."
1,"\n{\n ""CR"": ""\n### Motif d’hospitalisation\nB..."
2,"\n{\n ""CR"": ""\n## Compte rendu opératoire\n\n..."
3,"\n{\n ""CR"": ""\n## Compte rendu opératoire\n\n..."
4,"\n{\n ""CR"": ""\n**Hôpital Bichat-Claude Bernar..."
5,"\n{\n ""CR"": ""\n**Hôpital Nord - Assistance Pu..."
6,"\n{\n ""CR"": ""\n### Motif d’hospitalisation\nM..."
7,"\n{\n ""CR"": ""\n**Hôpital Michallon - CHU Gren..."
8,"\n{\n ""CR"": ""\n**SERVICE DE PHYSIOLOGIE**\n**..."
9,"\n{\n ""CR"": ""\n**SERVICE D’ORL INFANTILE**\n*..."
